<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-395/477 Redes Neuronales Artificiales I-2018 </h1>

<H3 align='center'> Tarea 2 - Aplicaciones Recientes de Redes Neuronales </H3>
<hr style="height:2px;border:none"/>

**Temas**  

* Diseño e implementación detallado de Redes Recurrentes (RNN) y sus derivados.
* Compuertas LSTM y GRU. 
* Arquitectura Encoder-Decoder para problemas de *traducción*.
* Arquitectura de Autoencoder para reducción de dimensionalidad.
* Modelos generativos profundos: VAEs (*Variational Autoencoder*)

** Formalidades **  
* Equipos de trabajo de: 2-3 personas (*cada uno debe estar en condiciones de realizar una presentación y discutir sobre cada punto del trabajo realizado*)
* Se debe preparar una presentación de 20 minutos. Presentador será elegido aleatoriamente.
* Se debe preparar un (breve) Jupyter/IPython notebook que explique la actividad realizada y las conclusiones del trabajo
* Fecha de entrega y discusión: por definir.
* Formato de entrega: envı́o de link Github al correo electrónico del ayudante (*<francisco.mena.13@sansano.usm.cl>*) , incluyendo al profesor en copia (*<jnancu@inf.utfsm.cl>*). Por favor especificar el siguiente asunto: [Tarea2-INF395-I-2019]

<hr style="height:2px;border:none"/>
La tarea se divide en secciones:

[0+.](#bonus) Bonus  
[1.](#primero) RNN sobre texto  
[2.](#segundo) *Encoder-Decoder* sobre imágenes    
[3.](#tercero) *Encoder-Decoder* sobre Texto    
[4.](#cuarto) Distintos tipos de autoencoders (AEs) en MNIST    

<a id="bonus"></a>
## 0+. Bonus: Predicción de Ventas en Falabella
---

El problema de esta sección es muy similar al problema 4 de la tarea anterior y les permitirá corregir o entrenarse mejor en aquello que no hayan aprendido bien antes. Esta vez trabajaremos con series de datos reales correspondientes a las ventas de una conocida empresa nacional de retail (Falabella). Los datos fueron recolectados, pre-procesados y anonimizados por una tesista de esta universidad, a través del siguiente __[link](https://www.dropbox.com/sh/14or2wjdb3hp45l/AAAFx325iQ1OlLHacFB3sCh4a?dl=0)__.

<img src="https://www.modaes.com/files//000_2016/falabella/falabella-entrada-exterior-parque-arauco-728.jpg" width="50%" />



Grosso modo, la tarea consiste en anticipar las ventas de los diferentes productos de la compañía con al menos 1 mes de anticipación, es decir predecir hacia "adelante" desde al menos 1 mes. Para simplificar, nos concentraremos en solo **una** de las múltiples tiendas existentes y trataremos de predecir 4 productos de los 400.000 que maneja la empresa. Sin embargo, dispondremos de las series de ventas de todos los 400.000 productos para usar como predictores. Los datos disponibles cubren un horizonte de dos años y tienen frecuencia diaria, es decir, las series tiene un largo total de aproximadamente 2*350 = 700 valores. 

El formato del archivo de datos viene descrito en el archivo *README*. Cada fila corresponde a un día. En cada fila/día se representan las ventas de cada producto. Los productos no vendidos (venta=0) no aparecen.

Como metadata adicional (a usar de modo completamente libre y voluntario), se tiene: un árbol que indica las categorías a las que pertenecen los 400.000 productos (por ejemplo: “zapatos-mujer”), las categorías a las que pertenecen estas categorías (por ejemplo “zapatos”), las categorías a las que pertenecen estas categorías (por ejemplo “moda”) y así sucesivamente. También se tiene un diccionario, que relaciona los ID de producto con marca y otras características. 

Esta actividad es completamente libre, sin código de ayuda, que sin embargo puede reciclar de la tarea anterior. 

> ### Si decide realizar la actividad 0+ (Bonus), el puntaje obtenido reemplazará la nota de su peor sección de la Tarea 1. Además de que también será válida como sección de ésta tarea, por lo que sólo debe realizar 3 de las siguientes 4 secciones.

<a id="primero"></a>
## 1. RNN sobre texto
---

Hoy en dı́a, una aplicación relevante de las redes neuronales recurrentes es el modelamiento de texto y lenguaje natural. En esta sección abordaremos el problema de procesar sentencias de texto, proporcionadas por GMB (*Groningen Meaning Bank*), para reconocimiento de entidades y *tagger*. En específico, trabajaremos con el dataset proprocionado a través de __[Kaggle](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus)__, que está compuesto por más de un millón de palabras, a fin de realizar predicciones sobre distintas tareas de redes recurrentes.

<img src="https://i.stack.imgur.com/b4sus.jpg" width="70%" />


Descargue los datos de la página de __[Kaggle](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus)__ y cárguelos mediante *pandas*.
```python
import numpy as np
import pandas as pd
df_ner = pd.read_csv("./entity-annotated-corpus/ner.csv", encoding ="cp1252", error_bad_lines=False)
df_ner.dropna(inplace=True)
```


> a) En esta primera instancia trabajaremos con la tarea de realizar un NER *tag* (**Named Entity Recognition**) sobre cada una de las palabras en las sentencias que se nos presenta en los datos. Esta tarea es del tipo *many to many*, es decir, la entrada es una secuencia y la salida es una secuencia, sin *shift*, por lo que necesitaremos una estructura de red adecuada a ésto. En primer lugar extraiga las columnas que utilizaremos del dataset **¿Por qué es conveniente utilizar *lemma* en vez de la misma palabra?**
```python
dataset = df_ner.loc[:,["lemma","word","pos","tag","prev-iob"]]
```
Luego de esto cree una estructura que contendrá todas las sentencias u oraciones (lista de *lemmas*) y otra estructura que contendrá las etiquetas (lista de *tags*). **¿Cuales son las dimensiones de ambas estructuras? ¿Cada dato de ejemplo tiene las mismas dimensiones que el resto?**
```python
n_used = 500000 #data to use-- your choice
dataX_raw,dataY_raw = [],[]
lemmas,labels = set(), set()  #uniques
for fila in dataset.values[:n_used]:
    if fila[-1]=="__START1__": 
        dataX_raw.append(sentence)
        dataY_raw.append(labels_sentence)
        sentence= []
        labels_sentence = []
    lemmas.add(fila[0])
    labels.add(fila[3])
    sentence.append(fila[0]) #add lemma
    labels_sentence.append(fila[3]) #TAG
dataX_raw = dataX_raw[1:]
dataY_raw = dataY_raw[1:]
```    

> b) Estudie la distribución del largo de los textos a procesar. Estudie también la frecuencia con la que aparecen las palabras en todo el dataset. **¿Se observa una ley Zipf?**[[1]](#refs) Realice un gráfico de la cantidad de datos por clase. Comente.


> c) Para representar cada posible *tag* y *lemma* de modo que la red pueda manejarlo, será necesario codificarlos a un número único (*indice*) ¿Cuántos *tags* y *lemmas* distintos existen?  Comente sobre el significado del *tag* para cada *lemma*. **Finalmente mida el largo máximo de entre todas las sentencias**.
```python
n_labels = len(labels)
lab2idx = {t: i for i, t in enumerate(labels)}
dataY = [[lab2idx[ner] for ner in ner_tags ] for ner_tags in dataY_raw] #Converting tags to indexs
n_lemmas = len(lemmas)
lemma2idx = {w: i for i, w in enumerate(lemmas)} 
dataX = [[lemma2idx[lemma] for lemma in sentence ] for sentence in dataX_raw] #Converting text to indexs
```

> d) Debido a la distinta extensión de textos se deberá **realizar *padding* para estandarizar el largo**,
considere algun carácter especial **no presente en el vocabulario** para codificar el espacio en blanco en ambos (entrada y salida), por ejemplo si el largo máximo es de 4 y se tiene la sentencia "the rocket" codificada como [32,4] será necesario agregar un *lemma* que codificado significará el fin de la sentencia "the rocket *ENDPAD ENDPAD*" que codificado quedará como [32,4,*N, N*]. Decida, respecto al cómo funciona una red recurrente y su *memoria*, sobre qué le parece más conveniente al momento de rellenar con un valor especial ¿Al principio o al final de la sentencia? Comente
```python
lemma2idx["END"] = n_lemmas #add fullfill lemma and tag to the dictionary
lab2idx["END"] = n_labels
n_labels +=1
n_lemmas +=1
from keras.preprocessing import sequence
X = sequence.pad_sequences(dataX, maxlen=max_input_lenght,padding='pre' or 'post',value=lemma2idx["yourspecialcharacter"])
y = sequence.pad_sequences(dataY, maxlen=max_input_lenght,padding='pre' or 'post',value=lab2idx["endtagger"])
del dataY[:],dataX[:]
```

> e) Para poder generar una representación adecuada sobre los datos de entrada que permita realizar operaciones lineales, deberá generar una representación a un vector denso. Para ésto se utilizará la arquitectura de autoencoder **Word2Vec** [[2]](#refs) sobre textos *raws* de largo variable, en donde el *encoder* codifica una palabra categórica (*target*) a un vector denso de dimensionalidad $d$ mientras que el *decoder* genera palabras en el contexto (*context*) de la palabra *target* (en una vecindad alrededor). La idea detrás es que palabras similares sean proyectadas a una región cercana en el espacio de *embedding* ¿Cuál es la importancia del parámetro min_count? ¿Cuántos *lemmas* ve Word2Vec?
```python
from gensim.models import Word2Vec
EMBEDDING_DIM = 32
window_size = 5
nb_epoch = 5
batch_size = 6000
min_count = 3
model = Word2Vec(dataX_raw,size=EMBEDDING_DIM,window=window_size,batch_words=batch_size,iter=nb_epoch,
                 min_count=min_count, negative=5,sg=1) #sg=1 mean skip-gram
embeddings_index = {vocab_word: model.wv[vocab_word] for vocab_word in model.wv.vocab}
len(embeddings_index.keys())
```
Genere una matriz de *embeddings* que se utilizarán como capa neuronal.
```python
embedding_matrix = np.zeros((n_lemmas, EMBEDDING_DIM))
for word, i in lemma2idx.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: #if word does not has embedding
        embedding_matrix[i] = embedding_vector
```
Luego, para poder realizar una clasificación sobre los datos en la salida será necesario representarlos *one hot vectors*, esto resultará en un arreglo tridimensional.
```python
from keras.utils import to_categorical
y = np.asarray([to_categorical(i, num_classes=n_labels) for i in y])
```

> f) Luego de esto cree los conjuntos de entrenamiento y de prueba con el código a continuación **¿Cuáles son las dimensiones de entrada y salida de cada conjunto?** Comente
```python
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=22)
y_train.shape
```

> g) Defina una red neuronal recurrente *many to many* con compuertas LSTM para aprender a *tagear* la entidad en el texto. Esta red debe procesar la secuencia de *lemmas* rellenados (o sin rellenar) y entregar el *tag* a cada uno de los *lemmas*, por lo que la salida de la red es una por cada instante de tiempo que se necesita entregar un *output*. La primera capa de la red a construir debe tener los vectores de *embedding* encontrados por **Word2Vec**. **Comente sobre los cambios que sufre un dato al ingresar a la red y la cantidad de parámetros de la red**. Entrene y luego evalúe su desempeño sobre ambos conjuntos. 
```python
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, Dropout,TimeDistributed
model = Sequential()
model.add(Embedding(input_dim=n_lemmas, output_dim=EMBEDDING_DIM, input_length=max_input_lenght,
                    trainable=False, weights = [embedding_matrix]))
model.add(LSTM(units=100,return_sequences=True))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(n_labels, activation='softmax')))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=128)
```
Para evaluar su modelo utilice una métrica adecuada para el desbalance presente entre las clases como identificó en el punto b). Tenga presente en este punto el **no evaluar** la clase/símbolo que añadió para realizar *padding* a los *tag*.  
*Hint: podría "truncar" la salida predicha hasta el largo real de esa sentencia*.
```python
from sklearn.metrics import f1_score
dataY_pred = model.predict_classes(X_test,verbose=0) #process... to remove prediction on "endtagger" symbol
f1_score_bydata = [f1_score(true, pred ,average='macro') for true,pred in zip(dataY,dataY_pred) ]
print("F1 score on test: ", np.mean(f1_score_bydata) )
```

> h) Varíe con seguir entrenando la capa de embedding seteada al definir la arquitectura, ésto es cambiar a *trainable=True*, compare el desempeño y el número de parámetros (entrenables) con lo anterior. Comente

> i) Experimente con cambiar la *gate* de recurrencia a una con menos parámetros pero que mantiene la capacidad de memoria de la LSTM, ésta es la compuerta GRU. Comente sobre los resultados esperados y observados.
```python
from keras.layers import GRU
...
model.add(GRU(units=100,return_sequences=True))
...
```

> j) Algunos autores señalan la importante dependencia que existe en texto, no solo con las palabras anteriores, sino que con las que siguen. Mejore la red utilizando una red neuronal recurrente Bidireccional, es decir, con recurrencia en ambas direcciones sobre la secuencia de *lemmas*. Comente cuál debiera ser la forma correcta de usar el parámetro merge_mode (concatenar, multiplicar, sumar o promediar) para este caso. Además comente las transformaciones que sufre el patrón de entrada al pasar por las capas. ¿Mejora o empeora el desempeño? Analice.
```python
from keras.layers import Bidirectional
...
gate_layer = (LSTM or GRU )(units=100,return_sequences=True)
model.add(Bidirectional(gate_layer, merge_mode=choose))
...
```

> k) En base a lo experimentado, **mejore el desempeño de las redes encontradas**, ya sea utilizando y/o combinando las distintas variaciones que se hicieron en los distintos ítemes, como bien alguna mejora en el pre-proceso de los datos (largo de secuencia, el tipo de *padding* o alguna otra), agregar mayor profundidad, variar el número de unidades/neuronas, utilizando otra *gate* de recurrencia (en https://keras.io/layers/recurrent/), cambiar los vectores de *embedding* por unos entrenados en otros dataset más grandes (https://nlp.stanford.edu/projects/glove/), entre otros.


> l) Utilice la red con mejor desempeño encontrada, idealmente la encontrada en (j), y **muestre las predicciones** del *NER tager*, sobre algún ejemplo de pruebas, comente.  
```python
p = model.predict(np.array([X_test[i]]))
p = np.argmax(p, axis=-1)
print("{:15}: {}".format("Lemma", "Pred"))
for w,pred in zip(X_test[i],p[0]):
    print("{:15}: {}".format(lemmas[w],labels[pred]))
```

<a id="segundo"></a>
## 2. Encoder-Decoder sobre imágenes
---
En la presente seccion se experimentará con arquitecturas del tipo *encoder-decoder* [[3]](#refs) aplicadas sobre imágenes, tales como *image translation*, *object location*, *image segmentation*, entre otros. La idea es aplicar una red convolucional en ambas partes del modelo (encoder y decoder), así utilizamos un modelo que se adapte a estos casos.

La tarea consistirá en realizar **Image Segmentation** [[4]](#refs) para identificar ciertos segmentos o regiones de interés en una imagen a través de procesar de manera semántica (en la codificación) si cada pixel corresponde a un segmento a destacar. Esta tarea puede ser aplicada tanto para identificar un segmento como para identificar múltiples segmentos a través de colocar varios canales/filtros de salida en el *decoder*. Para ésto trabajaremos con un dataset creado en el área (*A BENCHMARK FOR SEMANTIC IMAGE SEGMENTATION*). El dataset resulta bastante pequeño en cantidad de datos, por lo que deberá pensar en formas de conllevar ésto.

<img src="https://qph.fs.quoracdn.net/main-qimg-78a617ec1de942814c3d23dab7de0b24" width="70%" />

Descargue los datos a través del siguiente __[link](http://www.ntu.edu.sg/home/ASJFCai/Benchmark_Website/benchmark_index.html)__. Luego cargue las pocas imágenes a trabajar con la librería __[Pillow](pillow.readthedocs.io)__. Debido a la dimensionalidad variable de los datos de entrada deberá redimensionar a un valor que considere prudente, *se aconseja menos de 250*, comente su decisión.
```python
import numpy as np
import os
img_size = choose
folder = "imagefolder.."
data = [archivo.split(".")[0] for archivo in os.listdir(folder+"/image")]
from PIL import Image
X_image = []
for archivo in data:
    I = Image.open(folder+"/image/"+archivo+".jpg")
    I = np.asarray(I.resize( (img_size,img_size),Image.ANTIALIAS ))
    X_image.append(I)
X_image = np.asarray(X_image)
Y_image = []
for archivo in data:
    I = Image.open(folder+"/ground-truth/"+archivo+".png")
    I = np.asarray(I.resize( (img_size,img_size),Image.ANTIALIAS ))
    Y_image.append(I)
Y_image = np.asarray(Y_image)
```

> a) Explore los datos a trabajar, visualice la entrada y salida del modelo, además de las dimensionalidades de éstas ¿Es un problema las dimensiones de los datos *versus* la cantidad de datos a entrenar? Normalice los datos como se acostumbra en imágenes y genere una dimensión/canal extra a la salida.
```python
...#visualize and do nice plots!
X_image = X_image/255.
Y_image = Y_image/255.
Y_image = Y_image[:,:,:,None]
```

> b) Separe 10 imágenes como conjunto de pruebas para verificar la calidad del modelo entrenado.

> c) Debido a la poca cantidad de datos presentes defina la arquitectura a utilizando únicamente convolucionales (*fully convolutional*) [[5]](#refs), como la presente en el código. Comente sobre los cambios en la dimensionalidad a través del *forward pass*. Decida el tamaño del *batch* en base a la cantidad de datos que se presenta para entrenar.
```python
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D, Conv2DTranspose, UpSampling2D, BatchNormalization
model = Sequential()
...#ENCODER PART
model.add(Conv2D(32, (3, 3), activation='relu', padding='same',input_shape=X_image.shape[1:]))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
...#DECODER PART
model.add(UpSampling2D((2, 2)))
model.add(Conv2DTranspose(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2DTranspose(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2DTranspose(1, (3, 3), strides=(2,2), activation='sigmoid', padding='same')) #pixel-wise classification
model.summary()
model.compile(loss='binary_crossentropy',optimizer='rmsprop')
model.fit(X_train,Y_train,epochs=100,batch_size=...)
```

> d) Para medir el desempeño del modelo sobre ambos conjuntos realice un análisis cualitativo en base a visualizar la segmentación que realiza *versus* la segmentación real, además de verificar el *precision* y *recall* asumiendo valores binarios de pixel ¿Qué valor debería ser más importante los ceros o 1? Comente.
```python
Y_hat_train = np.squeeze( model.predict(X_train) )
Y_hat_test = np.squeeze( model.predict(X_test) )
...#visualice Y_hat and Y_image
from sklearn.metrics import precision_score, recall_score
Y_label = Y_train.flatten() >0.5
Y_hat_label = Y_hat_train.flatten() >0.5
print(precision_score(Y_label, Y_hat_label, average=None, labels=[0,1] ))
print(recall_score(Y_label, Y_hat_label, average=None , labels=[0,1]))
```

> e) Compárese con alguna técnica manual de *Image Segmentation*, comúnmente se sugiere considerar un *treshold* para activar o apagar un píxel. Experimente con utilizar *treshold* igual a la media o con otra técnica más inteligente basada en los histogramas de escala de grises, como se utilizan en *skimage*.
```python
gray_X = 0.2125*X_image[:,:,:,0]+ 0.7154*X_image[:,:,:,1]+ 0.0721*X_image[:,:,:,2] #needed gray-scale
...
""" One option"""
val = gray_X.mean()  #or another statistical
""" Another option"""
from skimage import filters
val = filters.threshold_otsu(gray_X)
... 
mask = gray_X < val
X_segmented = mask*1
```

> f) Experimente con realizar *data augmentation* sobre el problema. Debido a que las operaciones clásicas de *augmentation* como rotar, invertir, girar, cambiarian la etiqueta de segmentación, genere una estrategia que mantenga la etiqueta/salida $Y$. Se presenta un código de ejemplo, *Denoising*, de aplicar una máscara binaria aleatoria sobre la imagen de entrada $X$, **de todas formas se espera que proponga alguna distinta**. Compare el desempeño alcanzado con la nueva red con la forma de evaluar definida en (d).
```python
from numpy.random import binomial #DENOISING IDEA
T = 100
for _ in range(T):
    noise_level = np.random.randint(4,10)/10.
    noise_mask = binomial(n=1,p=noise_level,size=X_image.shape)
    X_augmented = X_image*noise_mask
    model.fit(X_augmented,Y_image,epochs=1,batch_size=32,validation_data=(X_image,Y_image))
```

> g) Intente variar la arquitectura presentada en pos de obtener un mejor modelo, basado en la evaluación realizada en (d). Recuerde tomar en cuenta la poca cantidad de datos que se tiene.

<a id="tercero"></a>
## 3. *Encoder-Decoder* sobre Texto

Trabajos recientes en redes neuronales han demostrado que se puede aplicar a problemas bastante complejos gracias a la flexibilidad la definición de las redes, además de que se pueden adaptar a distintos tipos de datos brutos (dominios). Con el objetivo de explorar el enfoque anterior de *traducción* de algun tipo de dato, en esta sección deberá realizarlo con texto para traducción de un lenguaje humano a otro (e.g. inglés a alemán, chino a ruso).

<img src="https://www.panoramaila.cz/images/preklady.jpg" width="35%" />


Trabajaremos con el dataset de pares de sentencias bilingues entre distintos idiomas del proyecto __[Tatoeba](http://www.manythings.org/anki/)__. El objetivo entonces consta de tomar un texto en lenguaje natural de algún idioma (*source*) y traducirlo a otro texto en lenguaje natural de otro idioma (*target*), donde cada texto tendrá un largo variable. Lo cual se empleará a través de extrar información del texto *source* (*encoder*) para luego generar el texto *target* (*decoder*) en base a esta información extraída.


Deberá seleccionar el *dataset* que guste para trabajar con la tarea de traducción, comente sobre su decisión. Luego cárgelo con *pandas*.
```python
import pandas as pd
df = pd.read_csv("data/dataset_selected.txt", sep="\t", names=["Source","Target"])
df.head()
```

> a) Visualice los datos ¿Qué es la entrada y qué es la salida? Comente sobre los múltiples significados/sinónimos que puede tener una palabra al ser traducida y cómo propondría arreglar eso. *se espera que pueda implementarlo*

> b) Realice un pre-procesamiento a los textos como se acostumbra para eliminar símbolos inecesarios u otras cosas que estime conveniente, comente sobre la importancia de éste paso. Además de ésto deberá agregar un símbolo al final de la sentencia *target* para indicar un "alto" cuando la red neuronal necesite aprender a generar una sentencia.
```python
import string
table = str.maketrans('', '', string.punctuation) 
def clean_text(text, where=None):
    """ OJO: Sin eliminar el significado de las palabras."""
    text = text.lower()
    tokenize_text = text.split()
    tokenize_text = [word.translate(table) for word in tokenize_text]#eliminar puntuacion
    tokenize_text = [word for word in tokenize_text if word.isalpha()] #remove numbers
    if where =="target":
        tokenize_text = tokenize_text + ["#end"] 
    return tokenize_text
texts_input = list(df['Source'].apply(clean_text))
texts_output = list(df['Target'].apply(clean_text, where='target'))
```

> Cree un conjunto de validación y de pruebas fijos de $N_{exp} = 10000$ datos ¿Cuántos datos quedan para entrenar? 
```python
from sklearn.model_selection import train_test_split
X_train_l, X_test_l, Y_train_l, Y_test_l = train_test_split(texts_input, texts_output,
                                                            test_size=N_exp, random_state=22)
X_train_l, X_val_l, Y_train_l, Y_val_l = train_test_split(X_train_l, Y_train_l, 
                                                          test_size=N_exp, random_state=22)
```
*Recuerde que si no puede procesar los datos de entrenamiento adecuadamente siempre puede muestrear en base a la capacidad de cómputo que posea*

> c) Genere un vocabulario, **desde el conjunto de entrenamiento**, sobre las palabras a recibir y generar en la traducción, esto es codificarlas a un valor entero que servirá para que la red las vea en una representación útil a procesar, *comience desde el 1 debido a que el cero será utilizado más adelante*. Para reducir el vocabulario considere las palabras que aparecen un mínimo de *min_count* veces en todo los datos, se aconseja un valor de 3. Comente sobre la importancia de ésto al reducir el vocabulario ¿De qué tamaño es el vocabulario de entrada y salida? ¿La diferencia de ésto podría ser un factor importante?
```python
def create_vocab(texts, min_count=1):
    count_vocab = {}
    for sentence in texts:
        for word in sentence:
            if word not in count_vocab:
                count_vocab[word] = 1
            else:
                count_vocab[word] += 1
    return [word for word,count in count_vocab.items() if count >= min_count]
vocab_source = create_vocab(X_train_l, min_count=3)
word2idx_s = {w: i+1 for i, w in enumerate(vocab_source)} #index (i+1) start from 1,2,3,...
idx2word_s = {i+1: w for i, w in enumerate(vocab_source)}
n_words_s = len(vocab_source)
vocab_target = create_vocab(Y_train_l, min_count=3)
word2idx_t = {w: i+1 for i, w in enumerate(vocab_target)}  #Converting text to numbers
idx2word_t = ... #Converting number to text
n_words_t = ...
```
Ahora codifique las palabras a los números indexados con el vocabulario. Recuerde que si una palabra en los otros conjuntos, o en el mismo de entrenamiento, no aparece en el vocabulario no se podrá generar una codificación, por lo que será **ignorada** ¿Cómo se podría evitar ésto?
```python
""" Source/input data """
dataX_train = [[word2idx_s[word] for word in sent if word in word2idx_s] for sent in X_train_l]
dataX_valid = [[word2idx_s[word] for word in sent if word in word2idx_s] for sent in X_val_l]
... #do test
""" Target/output data """
dataY_train = [[word2idx_t[word] for word in sent if word in word2idx_t] for sent in Y_train_l]
dataY_valid = [[word2idx_t[word] for word in sent if word in word2idx_t] for sent in Y_val_l] 
...#do test
```

> d) Debido al largo variable de los textos de entrada y salida será necesario estandarizar ésto para poder trabajar de manera más cómoda en Keras, *cada texto (entrada y salida) pueden tener distinto largo máximo*. Comente sobre la decisión del tipo de *padding*, *pre o post* ¿Qué sucede al variar el largo máximo de instantes de tiempo para procesar en cada parte del modelo (entrada y salida)?
```python
from keras.preprocessing import sequence
""" INPUT DATA (Origin language) """
max_inp_length = max(map(len,dataX_train))
print("Largo max inp: ",max_inp_length)
word2idx_s["*"] = 0 #padding symbol
idx2word_s[0] = "*"
n_words_s += 1  
X_train = sequence.pad_sequences(dataX_train, maxlen=max_inp_length, padding='pre', value=word2idx_s["*"])
...# do valid and test..
""" OUTPUT DATA (Destination language) """
max_out_length = max(map(len,dataY_train)) 
print("Largo max out: ",max_out_length)
word2idx_t["*"] = 0 #padding symbol
idx2word_t[0] = "*"
n_words_t += 1  
Y_train = sequence.pad_sequences(dataY_train, maxlen=max_out_length, padding='post', value=word2idx_t["*"])
...#do valid and test..
```

> e) Para evitar que la red obtenga una ganancia por imitar/predecir el símbolo de *padding* que está bastante presente en los datos coloque un peso sobre éste clase, con valor 0, así se evita que tenga impacto en la función objetivo. Ya que *keras* no soporta directamente ésto en series de tiempo coloque el peso a cada instante de tiempo de cada dato de entrenamiento dependiendo de su clase. Comente sobre alguna otra forma en que se podría manejar el evitar que la red prediga en mayoría el símbolo de *padding*.
```python
c_weights = np.ones(n_words_t)
c_weights[0] = 0 #padding class masked
sample_weight = np.zeros(Y_train.shape)
for i in range(sample_weight.shape[0]):
    sample_weight[i] = c_weights[Y_train[i,:]]
```

> f) Para lograr la tarea defina una red recurrente del tipo *encoder*-*decoder* como la que se presenta en la siguiente imágen.
<img src="https://chunml.github.io/ChunML.github.io/images/projects/sequence-to-sequence/repeated_vector.png" width="60%" />
En primer lugar defina el *Encoder* que procesara el texto de entrada y retornará un solo vector final, haciendo uso de las capas ya conocidas de *Embedding* para generar un vector denso de palabra y *GRU*, pero en su versión acelerada para GPU.
```python
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU
EMBEDDING_DIM = 100
model = Sequential()
model.add(Embedding(input_dim=n_words_s, output_dim=EMBEDDING_DIM, input_length=max_inp_length))
model.add(CuDNNGRU(64, return_sequences=True))
model.add(CuDNNGRU(128, return_sequences=False))
```
Luego defina la sección que conecta el largo (*timesteps*) de entrada *vs* el de salida.
```python
from keras.layers import RepeatVector
model.add(RepeatVector(max_out_length)) #conection
```
Finalmente defina el *Decoder* para generar la secuencia de salida en texto de palabras en otro idioma, a través de la función *softmax* sobre cada instante de tiempo (*timestep*. 
```python
from keras.layers import CuDNNGRU, TimeDistributed,Dense
model.add(CuDNNGRU(128, return_sequences=True))
model.add(CuDNNGRU(64, return_sequences=True))
model.add(TimeDistributed(Dense(n_words_t, activation='softmax')))
model.summary()
```
Entrene la red entre 1 a 5 *epochs*, agregando los pesos definidos sobre cada ejemplo de entrenamiento. Además de utilizar una función de pérdida que evita generar explícitamente los *one hot vector*
```python
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', sample_weight_mode='temporal')
model.fit(X_train, Y_train, epochs=3, batch_size=256,validation_data=(X_valid, Y_valid),
         sample_weight = sample_weight) 
```

> g) Debido a lo costoso de tener una red completamente recurrente para entrenar y poder experimentar, cambie el modelo que procesa el *Encoder* por una red convolucional, reduciendo el número de capas pero aumentando las neuronas. Utilice tamaños de *kernel*  igual a 5 y funciones de activaciones relu. Se agregan capas de *BatchNormalization* debido a que en el *Decoder* contamos con redes recurrentes que tienen capa activación distinta a la usada por las convoluciones. La capa de *GlobalMaxPooling1d* es lo que permite reducir toda la información extraída a un único vector, como se realizó anteriormente con *return_sequences=False*, comente sobre la ganancia o desventaja de ésto *vs* la red neuronal.
```python
from keras.layers import Conv1D,MaxPool1D,GlobalMaxPooling1D,GlobalAveragePooling1D,BatchNormalization
model = Sequential()
model.add(Embedding(input_dim=n_words_s, output_dim=EMBEDDING_DIM, input_length=max_inp_length))
model.add(Conv1D(256, 5, padding='same', activation='relu', strides=1))
model.add(BatchNormalization()) #for stability
model.add(Conv1D(256, 5, padding='same', activation='relu', strides=1))
model.add(BatchNormalization())
model.add(GlobalMaxPooling1D()) #aka to return_sequences=False
model.add(RepeatVector(max_out_length)) #conection
model.add(CuDNNGRU(256, return_sequences=True))
model.add(TimeDistributed(Dense(n_words_t, activation='softmax')))
model.summary() 
```
Entrene el modelo igual a lo presentado anteriormente pero ahora por 20 *epochs* ¿Cambian los tiempos de procesamiento y la cantidad de parámetros?

> h) Visualice lo aprendido por el modelo sobre algunos datos del conjunto de entrenamiento y validación, comente lo observado.
```python
def predict_words(y_indexs, data="target"):
    """ Predict until '-#end-' is seen """
    return_val = []
    for indx_word in y_indexs:
        if indx_word != 0: #start to predict
            return_val.append(np.squeeze(indx_word))
            if data == "target": #if target is predicting
                if indx_word == word2idx_t["#end"]:
                    return return_val                
    return return_val
n_s = 100
idx = np.random.choice(np.arange(Y_set.shape[0]), size=n_s, replace=False)
Y_set_pred = model.predict_classes(X_set[idx] )
for i, n_sampled in enumerate(idx):
    text_input = [idx2word_s[p] for p in predict_words(X_set[n_sampled], data="source")]
    print("Texto source: ", ' '.join(text_input))
    text_real = [idx2word_t[p] for p in predict_words(Y_set[n_sampled,:,0], data="target")]
    print("Texto target real: ", ' '.join( text_real))
    text_sampled = [idx2word_t[p] for p in predict_words(Y_set_pred[i], data="target")]
    print("Texto target predicho: ", ' '.join(text_sampled))
```

> i) Realice algún cambio esperando que mejore el modelo entrenado, luego vuelva a visualizar lo predicho por la red *vs* lo real. *Debido a lo costoso en entrenar puede optar por realizar solo un cambio pero que sea significativo*.  Se comentan algunas opciones para utilizar y combinar:
* Cambiar  el *embedding* por alguno pre-entrenado
* Agregar regularizadores
* Asignar peso a las clases/palabras de salida
* Cambiar *Global max pooling* por *Average max pooling*
* Aumentar o reducir capas
* Aumentar o reducir neuronas/unidades  

> j) A pesar de que la tarea de medir qué tan similar es un texto a otro ya es un área de investigación propia [[6]](#refs), usted deberá utilizar alguna métrica de desempeño para ver qué tan buena es la traducción del texto *versus* el texto real entregado. Debido a que la métrica de *Exact Matching* (EM) puede ser muy drástica, mida *f1 score* por texto además de proponer alguna otra técnica de evaluación para medir sobre el conjunto de pruebas y los otros conjuntos si estima conveniente. Puede basarse en otros trabajos como *Image captioning* o *Text summary*. 
*Hint: Debido a los problemas de memoria al realizar un forward-pass, solo seleccione un subconjunto $N_{sub}$ del conjunto de pruebas para realizar ésta evaluación, se aconseja entre 1000 y 5000.*
```python
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, precision_score, recall_score
m = MultiLabelBinarizer().fit([np.arange(n_words_t)]) 
def calculate_f1(true, pred):
    true = np.squeeze(true)
    pred = np.squeeze(pred)
    binarized_true = m.transform([predict_words(true)])[0] #onehot of words appear
    binarized_pred = m.transform([predict_words(pred)])[0] #onehot of words appear
    return f1_score(binarized_true, binarized_pred, average='binary') #only on appearing words
f1_final = np.mean([calculate_f1(true_words,pred_words) for true_words,pred_words in zip(Y_true,Y_hat)])
f1_final*100 #porcentaje
```
>> La función de *f1 score* en este extracto se calcula en base al *precision* y *recall* de que aparezca cada una de las palabras predichas dentro de las palabras reales (como si cada palabra fuera una clase de "aparece" o no), **sin importar el orden ni la ocurrencia**.


> k) En ves de volver a variar el modelo de *Encoder*, dejaremos una representación manual explícita (*no entrenable*) a través de extraer características manuales de los textos *source*, como por ejemplo representaciones *term frequency* (TF) o TF-IDF, proporcionadas a través de __[sklearn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text)__. Luego, con esto generado, defina y entrene el modelo *Decoder* neuronal como el presentado en las preguntas anteriores, ésto es comenzar desde la capa *RepeatVector* hasta llegar a la clasificación sobre el texto *target*. Compare el desempeño con lo presentado en (j) y lo visualizado en (h).
```python
from sklearn.feature_extraction.text import TfidfVectorizer
def dummy_fun(doc):
    return doc
tf_idf = TfidfVectorizer(analyzer='word',tokenizer=dummy_fun,preprocessor=dummy_fun,
                         token_pattern=None,use_idf= True, smooth_idf=True, norm='l2')   
X_train_tfidf = tf_idf.fit_transform(dataX_train).astype('float32').todense()
...#do over valid and test..
```

<a id="cuarto"></a>
## 4. Distintos tipos de autoencoders (AEs) en MNIST
---

Como se ha discutido en clases, las RBM’s y posteriormente los AE's (redes no supervisadas) fueron un componente crucial en el desarrollo de los modelos que entre 2006 y 2010 vigorizaron el área de las redes neuronales artificiales con logros notables de desempeño en diferentes tareas de aprendizaje automático. Recientemente se ha propuesto AE's con distribuciones de probabilidades en su codificación, VAE [[7]](#refs). Los VAE son una variación bayesiana que aprende los parámetros de alguna distribución de probabilidad de variables latentes definida sobre los datos, a través de esa variable latente el decodificador generar/reconstruye nuevos datos $\hat{x}$. En resumidas cuentas, es un autoencoder que aprende un modelo sobre las variables latentes de los datos.

<img src="https://www.researchgate.net/profile/Steven_Young11/publication/306056875/figure/fig1/AS:393921575309346@1470929630835/Example-images-from-the-MNIST-dataset.png" title="mnist" width="25%" style="float: right;" />

Con este objetivo en mente, utilizaremos un dataset simple denominado **MNIST**. Se trata de una colección de 70000 imágenes de 28 $\times$ 28 pixeles correspondientes a dígitos manuscritos (números entre 0 y 9). En su versión tradicional, la colección se encuentra separada en dos subconjuntos: uno de entrenamiento de 60000 imágenes y otro de test de 10000 imágenes



Cargue los datos desde el repositorio de Keras.
```python
import numpy as np
import keras
import keras.backend as K
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train[:,:,:,None] #add channels
x_test = x_test[:,:,:,None]
img_rows, img_cols,channel = x_train.shape[1:]
original_img_size = (img_rows, img_cols,channel) # input image dimensions
```

> a) Normalice las imágenes de modo que los pixeles queden en el rango [0, 1] como se acostumbra al trabajar con imágenes. Visualice y comente sobre los datos a trabajar.
```python
x_train = x_train.astype('float32') / 255. 
...#and x_test
```

### 4.1 Autoencoder clásico
Una de las aplicaciones tı́picas de un AE es **reducción de dimensionalidad**, es decir, implementar una transformación $\phi:{\rm I\!R}^d \rightarrow {\rm I\!R}^{d'}$ de objetos representados originalmente por $d$ atributos en una nueva representación de $d'$ atributos, de modo tal que se preserve lo mejor posible la “información” original. Obtener tal representación es útil desde un punto de vista computacional (compresión) y estadı́stico (permite construir modelos con un menor número de parámetros libres). Un AE es una técnica de reducción de dimensionalidad no supervisada porque no hace uso de información acerca de las clases a las que pertenecen los datos de entrenamiento.


<img src="https://miro.medium.com/max/1400/0*yGqTBMopqHbR0fcF." title="AE" width="50%" />


> a) Entrene un AE básico, 1 capa escondida *feed forward*, para generar una representación de MNIST en $d'= 2, 8, 16, 32$ dimensiones. Justifique la elección de la función de pérdida a utilizar y del criterio de entrenamiento en general. Determine el porcentaje de compresión obtenido y el error de reconstrucción en cada caso. ¿Mejora el resultado si elegimos una función de activación **ReLU** para el Encoder? ¿Podrı́a utilizarse esta activación en el Decoder?
```python
from keras.layers import Input, Dense, Flatten,Reshape
from keras.models import Model
compres_dim = 2
input_img = Input(shape=original_img_size)
input_fl = Flatten()(input_img) #to get a vector representation
encoded = Dense(compres_dim, activation='sigmoid')(input_fl)
decoded = Dense(np.prod(original_img_size), activation='sigmoid')(encoded)
decoded = Reshape(original_img_size)(decoded)
autoencoder = Model(inputs=input_img, outputs=decoded)
encoder = Model(inputs=input_img, outputs=encoded)
autoencoder.compile(optimizer='rmsprop', loss='binary_crossentropy')
autoencoder.fit(x_train,x_train,epochs=40,batch_size=32,validation_data=(x_test,x_test))
autoencoder.save('basic_autoencoder.h5')
```

> b) Compare visualmente la reconstrucción que logra hacer el autoencoder desde la representación en ${\rm I\!R}^{d'}$ para algunas imágenes del conjunto de pruebas. Determine si la percepción visual se corresponde con el error de reconstrucción observada. Comente.
```python
from keras.models import load_model
autoencoder = load_model('basic_autoencoder.h5')
decoded_test = autoencoder.predict(x_test)
encoded_test = encoder.predict(x_test)
import matplotlib.pyplot as plt
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28),cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_test[i].reshape(28, 28),cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()
```

> c) Para verificar la calidad de la representación obtenida, implemente el clasificador denominado $kNN$ (k-nearest neighbor): dada una imagen $x$, el clasificador busca las k = 10 imágenes de entrenamiento más similares (de acuerdo a una distancia, e.g. euclidiana) y predice como clase, la etiqueta más popular entre las imágenes cercanas. Mida el error, en conjunto de entrenamiento y pruebas, obtenido construyendo este clasificador sobre la data reducida a través del autoencoder comparando con la representación reducida obtenida vía PCA (una técnica clásica de reducción de dimensionalidad) utilizando el mismo número de dimensiones $d'$= 2, 8, 16, 32. Considere tanto el error de reconstrucción como el desempeño en clasificación, además de comparar los tiempos medios de predicción en ambos escenarios. 
```python
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
pca = PCA(n_components=compres_dim)
pca.fit(x_train)
pca_train = pca.transform(x_train)
pca_test = pca.transform(x_test)
encoded_train = encoder.predict(x_train)
encoded_test = encoder.predict(x_test)
clf = KNeighborsClassifier(10) #CLASIFICATION
clf.fit(pca_train, y_train)
print('Classification Accuracy PCA %.2f' % clf.score(pca_test,y_test))
clf = KNeighborsClassifier(10) #CLASIFICATION
clf.fit(encoded_train, y_train)
print('Classification Accuracy %.2f' % clf.score(encoded_test,y_test))
```


> d) Modifique el autoencoder básico construido en (a) para implementar un *deep autoencoder* (más de dos capas) haciendo uso de las capas convolucionales para trabajar sobre matrices. Comente cómo sufre las transformaciones el patrón de entrada. Demuestre experimentalmente que este autoencoder puede mejorar la compresión obtenida por PCA y por el obtenido en (a) utilizando el mismo número de dimensiones $d'$ . Considere en esta comparación tanto el error de reconstrucción como el desempeño en clasificación (vı́a kNN) de cada representación. Comente.
```python
from keras.layers import Input,Conv2D,Flatten,Dense,MaxPooling2D, UpSampling2D
latent_dim = 2
input_img = Input(shape=original_img_size)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
before_F_shape =  (x.shape[1].value, x.shape[2].value, x.shape[3].value)
x = Flatten()(x)
encoded = Dense(compres_dim, activation=...)(x)
x = Dense(np.prod(before_F_shape),activation='relu')(encoded)
x = Reshape(before_F_shape)(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='rmsprop', loss='binary_crossentropy')
autoencoder.summary()
autoencoder.fit(x_train,x_train,epochs=40,batch_size=32,validation_data=(x_test,x_test))
autoencoder.save('deep_autoencoder.h5')
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
pca = PCA(n_components=target_dim)
pca.fit(x_train)
```


> e) Elija algunas de las representaciones aprendidas anteriormente y visualı́celas usando la herramienta *TSNE* disponible en la librerı́a *sklearn*. Compare cualitativamente el resultado con aquel obtenido usando PCA con el mismo número de componentes.
```python
nplot=5000 #warning: mind your memory!
encoded_train = encoder.predict(x_train[:nplot])
from sklearn.manifold import TSNE
model = TSNE(n_components=2, random_state=0)
encoded_train = model.fit_transform(encoded_train)
plt.figure(figsize=(10, 10))
colors={0:'b',1:'g',2:'r',3:'c',4:'m',5:'y',6:'k',7:'orange',8:'darkgreen',9:'maroon'}
markers={0:'o',1:'+',2: 'v',3:'<',4:'>',5:'^',6:'s',7:'p',8:'*',9:'x'}
for idx in xrange(0,nplot):
    label = y_train[idx]
    line = plt.plot(encoded_train[idx][0], encoded_train[idx][1],
        color=colors[label], marker=markers[label], markersize=6)
pca_train = pca.transform(x_train)
encoded_train = pca_train[:nplot]
... #plot PCA
```

### 4.2 Variational Autoencoder tradicional
El enfoque optimizador de los VAE sobre los parámetros modelados $\theta$ (decoder) y $\phi$ (encoder) que se deriva consta de minimizar la reconstrucción de los datos (al igual que un autoencoder tradicional), en base a alguna medicicón de error (*mse* por ejemplo) además de tener un factor de regularización que se impone para que la distribución aprendida de las variables latentes sea similar alguna distribución deseada *a priori*.  

$$ Min \ \mathcal{L}(p_{\theta}(x\mid z), \ x)\ +\ KL( q_{\phi}(z\mid x) \mid \mid p_{\theta}(z))$$

Con $\mathcal{L}$ la función de pérdida de reconstrucción, $KL$ la *KL Divergence* [[8]](#refs), $q_{\phi}(z \mid x)$ la codificación del dato a la variable latente $z$, $p_{\theta}(x\mid z)$ la recontrucción de los datos a través de las variables latentes $z$ y  $p_{\theta}(z)$ una distribución *a priori* asignada. 

<img src="https://i.imgur.com/ZN6MyTx.png" title="VAE" width="60%" />

> a) Defina la sección del *encoder* del VAE como el que se muestra en el código, de 3 tandas convolucionales y una *fully conected*, con una distribución Normal de 2 componentes para las variables latentes, $z \sim \mathcal{N} (\mu, \sigma^2 )$. Describa la arquitectura utilizada.
```python
from keras.layers import Input,Conv2D,Flatten,Dense,MaxPool2D
from keras.models import Model
filters = 32 # number of convolutional filters to use
num_conv = 3 # convolution kernel size
intermediate_dim = 128
latent_dim = 2
x = Input(shape=original_img_size)
conv_1 = Conv2D(filters,kernel_size=num_conv,padding='same', activation='relu')(x)
conv_2 = Conv2D(filters,kernel_size=num_conv,padding='same', activation='relu')(conv_1)
conv_3 = Conv2D(filters*2, kernel_size=num_conv, padding='same', activation='relu', strides=2)(conv_2)
flat = Flatten()(conv_3)
hidden = Dense(intermediate_dim, activation='relu')(flat)
z_mean = Dense(latent_dim,activation='linear')(hidden)
z_log_var = Dense(latent_dim,activation='linear')(hidden)
encoder = Model(x, z_mean) # build a model to project inputs on the latent space
```

> b) Defina la sección del *decoder* del VAE como el que se muestra en el código, una tanda *fully conected* y 3 tandas de la operación inversa a una convolución (**Convolución transpuesta** [[9]](#refs)), comente cómo ésta trabaja y cómo funcionan los parámetros de *stride*.
```python
from keras.layers import Reshape,Conv2DTranspose,Activation
shape_before_flattening = K.int_shape(conv_3)[1:] # we instantiate these layers separately to reuse them later
decoder_hid = Dense(intermediate_dim, activation='relu')
decoder_upsample = Dense(np.prod(shape_before_flattening), activation='relu')
decoder_reshape = Reshape(shape_before_flattening)
decoder_deconv_1 = Conv2DTranspose(filters,kernel_size=num_conv, padding='same',strides=2,activation='relu')
decoder_deconv_2 = Conv2DTranspose(filters,kernel_size=num_conv,padding='same', activation='relu')
decoder_mean_squash = Conv2DTranspose(channel, kernel_size=num_conv,padding='same', activation='sigmoid')
```

> c) Defina la sección que conecta a estas dos partes a través de un muestreo explícito de la distribución Normal (con $\epsilon \sim \mathcal{N}(0,1)$ se tiene $g = \mu + \sigma \cdot \epsilon$), ésto es lo que lo hace que sea un enfoque probabilístico/bayesiano. Describa el modelo completo.
```python
def sampling(args):
    epsilon_std = 1.0
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),mean=0., stddev=epsilon_std)
    return z_mean + K.exp(z_log_var) * epsilon
from keras.layers import Lambda
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
hid_decoded = decoder_hid(z)
up_decoded = decoder_upsample(hid_decoded)
reshape_decoded =  decoder_reshape(up_decoded)
deconv_1_decoded = decoder_deconv_1(reshape_decoded)
x_decoded_relu = decoder_deconv_2(deconv_1_decoded)
x_decoded_mean_squash = decoder_mean_squash(x_decoded_relu)
vae_norm = Model(x, x_decoded_mean_squash) # instantiate VAE model
vae_norm.summary()
```

> d) Como la función objetivo es *customizada* deberemos definirla y poner una distribución a *priori* sobre las variables latentes, en este caso se tendrá como media un vector de ceros y la matriz de covarianza la matriz identidad $p_{\theta}(z) \sim N (\vec{0},I)$. Elija la función de pérdida para la reconstrucción. Comente porqué la *KL Divergence* podría funcionar como regularizador del criterio de entrenamiento obtenido.
```python
from keras import backend as K # Compute VAE loss
choised_loss =  keras.metrics.binary_crossentropy(K.flatten(x),K.flatten(x_decoded_mean_squash))
choised_loss =  keras.metrics.mean_squared_error(K.flatten(x),K.flatten(x_decoded_mean_squash))
reconstruction_loss = img_rows * img_cols * channel* choised_loss
kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1) #closed form
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae_norm.add_loss(vae_loss)
vae_norm.summary()
```

> e) Entrene el modelo definido con los datos de MNIST entre 20 a 30 *epochs* con el optimizador de *RMSprop* y tamaño de batch el que estime conveniente.
```python
batch_size = ...
epochs =  [20,30]
vae_norm.compile(optimizer='rmsprop')
vae_norm.fit(x_train,epochs=epochs, batch_size=batch_size,validation_data=(x_test, None))
```

> f) Visualice la representación codificada $z$ (variables latentes) de los datos en base a su media $\mu_i$, compare cualitativamente con la representación *TSNE* del AE tradicional. Además genere un histograma de la media y la varianza $\sigma_i^2$ de las dos componentes. Comente.
```python
x_test_encoded = encoder.predict(x_test, batch_size=batch_size)
import matplotlib.pyplot as plt
plt.scatter(x_test_encoded[:, 0], x_test_encoded[:, 1], c=y_test)
plt.colorbar()
plt.show() # display a 2D plot of the digit classes in the latent space
encoder_log_var = Model(x,z_log_var)
...#histogram
```

> g) Genere nuevos datos artificialmente a través del espacio de las variables latentes. Para esto deberá generar puntos linealmente separados por debajo de la distribución Normal. Comente qué significada cada eje en la imagen ¿Qué sucede más allá en el espacio del 90% confianza de las variables latentes? ¿Qué objetos se generan?
```python
decoder_input = Input(shape=(latent_dim,))
_hid_decoded = decoder_hid(decoder_input)
_up_decoded = decoder_upsample(_hid_decoded)
_reshape_decoded = decoder_reshape(_up_decoded)
_deconv_1_decoded = decoder_deconv_1(_reshape_decoded)
_x_decoded_relu = decoder_deconv_2(_deconv_1_decoded)
_x_decoded_mean_squash = decoder_mean_squash(_x_decoded_relu)
generator = Model(decoder_input, _x_decoded_mean_squash) 
n = 30  # figure with 15x15 images 
image_size = img_cols
figure = np.zeros((image_size * n, image_size * n))
from scipy.stats import norm
grid_x = norm.ppf(np.linspace(0.05, 0.95, n)) #metodo de la transformada inversa
grid_y = norm.ppf(np.linspace(0.05, 0.95, n))
for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])            
        x_decoded = generator.predict(z_sample,batch_size=batch_size)
        figure[i * image_size: (i + 1) * image_size,
               j * image_size: (j + 1) * image_size] = x_decoded[0][:,:,0]
plt.figure(figsize=(10, 10))
plt.imshow(figure,cmap='gnuplot2')
pos = np.arange(image_size/2,image_size*n,image_size)
plt.yticks(pos,np.round(grid_y,1))
plt.xticks(pos,np.round(grid_x,1))
plt.show()
grid = norm.ppf(np.linspace(0.000005, 0.999995, n)) #en los extremos del intervalo de confianza
```

> h) Experimente y comente si mejora o empeora el desempeño de clasificación de la representación encontrada al aumentar la dimensionalidad de las variables latentes $z$, contrarrestándolo con el AE tradicional ($d' = 2,8,16,32 $). Explique.


### 4.3 Variational Autoencoder categórico

En esta última sección se explorará el caso en que se cambia el modelamiento sobre la variable latente a una distribuida Multinomial para representar una variable **categórica** que podría entregarnos cierta intuición de capturar las clases del problema de manera no supervisada. Para éste objetivo definiremos el número de variables latentes iguales a la cantidad de clases que sospechamos (en este caso son conocidas y corresponden a 10 dígitos).


> a) En primer lugar deberá definir la arquitectura realizando unos cambios leves a la presentada anteriormente. Comente las diferencias sobre los parámetros obtenidos.

> El primer cambio es en la distribución obtenida en el encoder.
```python
... #traditional VAE code here
latent_dim = 10
logits_z = Dense(latent_dim,activation='linear')(hidden) #log(p(z))
encoder = Model(x, logits_z) # build a model to project inputs on the latent space
```

> Luego, con el decoder creado (igual al caso anterior) es necesario cambiar la forma en que se conectan, ya que el muestreo ahora será a través de un truco diferente para variables categoricas (**Gumbel-Softmax[[10]](#refs))**.
```python
def sample_gumbel(shape,eps=K.epsilon()):
    """Inverse Sample function from Gumbel(0, 1)"""
    U = K.random_uniform(shape, 0, 1)
    return - K.log( -K.log(U + eps) + eps)
def sampling(logits_z):
    """ Perform a Gumbel-Softmax sampling"""
    tau = K.variable(2/3, name="temperature") 
    z = logits_z + sample_gumbel(K.shape(logits_z)) # logits + gumbel noise
    return keras.activations.softmax( z/tau )    
from keras.layers import Lambda
z = Lambda(sampling, output_shape=(latent_dim,))(logits_z)
... #traditional VAE code here
vae_norm = Model(x, x_decoded_mean_squash) # instantiate VAE model
vae_norm.summary()
```

> Finalmente la función de pérdida KL cambia ya que se asume un *prior* Multinomial con probabilidad uniforme, $p_{\theta}(z) = \frac{1}{K}$. ¿Qué interpretación se le da a este regularizador?
```python
... #traditional VAE code here
dist =  keras.activations.softmax(logits_z) # =p(z)
dist_neg_entropy = K.sum(dist * K.log(dist + K.epsilon()), axis=1)
kl_disc_loss =  np.log(latent_dim) + dist_neg_entropy #discrete KL-loss
vae_loss = K.mean(reconstruction_loss + kl_disc_loss)
vae_norm.add_loss(vae_loss)
... #traditional VAE code here
```

> b) Entrene el VAE categórico de la misma manera que realizó con el VAE tradicional en (e) ¿Nota algún cambio en este paso?.

> c) Para ver la efectividad del encoder en lograr capturar las clases es necesario medir una métrica de desempeño, sin embargo, las métricas clásicas como *accuracy* o *f1 score* no corresponderían a este caso debido a que las categoría capturada por el encoder no debería estar en el mismo orden de la clase real, ya que fue un entrenamiento no supervisado ¿Cómo se podría cambiar ésto?. Con esto en mente mida alguna métrica de __[*clustering*](https://scikit-learn.org/stable/modules/clustering.html)__ [[11]](#refs) sobre las categorías inferidas por el VAE. Comente.  
*Recuerde que se predice los logits de la probabilidad*
```python
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - x.max(axis=-1,keepdims=True) )
    return e_x / e_x.sum(axis=-1, keepdims=True)
p_z_train = softmax(encoder.predict(x_train))
p_z_test = softmax(encoder.predict(x_test))
y_train_pred = p_z_train.argmax(axis=-1)
y_test_pred = p_z_test.argmax(axis=-1)
...#Example
from sklearn.metrics import normalized_mutual_info_score
print(normalized_mutual_info_score(y_train, y_train_pred))
print(normalized_mutual_info_score(y_test, y_test_pred))
```

> d) Para entender mejor las categorías inferidas por el VAE genere datos "activando" una categoría y luego realizando un *forward pass* sobre el decoder/generador. Comente cualitativamente.
```python
decoder_input = Input(shape=(latent_dim,))
_hid_decoded = decoder_hid(decoder_input)
_up_decoded = decoder_upsample(_hid_decoded)
_reshape_decoded = decoder_reshape(_up_decoded)
_deconv_1_decoded = decoder_deconv_1(_reshape_decoded)
_x_decoded_relu = decoder_deconv_2(_deconv_1_decoded)
_x_decoded_mean_squash = decoder_mean_squash(_x_decoded_relu)
generator = Model(decoder_input, _x_decoded_mean_squash) 
predictions =np.zeros((img_cols * 1 ,img_cols* latent_dim))
for i in range(latent_dim):
    activate_aux = np.zeros((1,10))
    activate_aux[:,i] = 1 #activate a class
    predictions[:,i * img_cols:(i + 1) * img_cols] = np.squeeze(generator.predict(activate_aux))
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
plt.imshow(predictions, cmap='gnuplot2')
pos = np.arange(img_cols/2, img_cols*latent_dim, img_cols)
plt.xticks(pos,range(latent_dim))
plt.show()
```

> e) Genere algunas imágenes aleatorias, comente cualitativamente con lo obtenido con el VAE tradicional ¿Cuál pareciera ser mejor para generar datos? ¿Por qué?

<a id="refs"></a>
## Referencias
[1] https://es.wikipedia.org/wiki/Ley_de_Zipf    
[2] Mikolov, T., Chen, K., Corrado, G., & Dean, J. (2013). *Efficient estimation of word representations in vector space*. arXiv preprint arXiv:1301.3781.    
[3] Cho, K., Van Merriënboer, B., Bahdanau, D., & Bengio, Y. (2014). *On the properties of neural machine translation: Encoder-decoder approaches*. arXiv preprint arXiv:1409.1259.  
[4] Pal, N. R., & Pal, S. K. (1993). *A review on image segmentation techniques*. Pattern recognition, 26(9), 1277-1294.  
[5] Badrinarayanan, V., Kendall, A., & Cipolla, R. (2017). *Segnet: A deep convolutional encoder-decoder architecture for image segmentation*. IEEE transactions on pattern analysis and machine intelligence, 39(12), 2481-2495.  
[6] Papineni, K., Roukos, S., Ward, T., & Zhu, W. J. (2002, July). *BLEU: a method for automatic evaluation of machine translation*. In Proceedings of the 40th annual meeting on association for computational linguistics (pp. 311-318). Association for Computational Linguistics.  
[7] Kingma, D. P., & Welling, M. (2013). *Auto-encoding variational bayes*. arXiv preprint arXiv:1312.6114.  
[8] https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence     
[9] Dumoulin, V., & Visin, F. (2016). *A guide to convolution arithmetic for deep learning*. arXiv preprint arXiv:1603.07285.  also: https://github.com/vdumoulin/conv_arithmetic  
[10] Jang, E., Gu, S., & Poole, B. (2016). *Categorical reparameterization with gumbel-softmax*. arXiv preprint arXiv:1611.01144.  
[11] https://en.wikipedia.org/wiki/Cluster_analysis  